In [33]:
# Import Libraries
import os
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from PIL import Image
import glob
import matplotlib.pyplot as plt
from random import shuffle
import pandas as pd


In [37]:
# Load cone_data
file_path1 = r'C:\Users\WeihaoFan\data\CNNreconstruct\Dataset_CNNreconstruct_1.xlsx'
file_path2 = r'C:\Users\WeihaoFan\data\CNNreconstruct\Dataset_cm_CNNreconstruct.xlsx'
exnf = pd.read_excel(file_path1, sheet_name='Sheet1')
exn = pd.read_excel(file_path1, sheet_name='Sheet2')
cmtypes = pd.read_excel(file_path2, sheet_name='Sheet1')
cmpositions = pd.read_excel(file_path2, sheet_name='Sheet2')
cmspcings = pd.read_excel(file_path2, sheet_name='Sheet3')
cmspertures = pd.read_excel(file_path2, sheet_name='Sheet4')
mask = pd.read_excel(file_path2, sheet_name='Sheet5')


In [45]:
df = pd.read_csv(r'C:\Users\WeihaoFan\data\CNNreconstruct\zebra.csv')

df

,195,132,164,38,126,179,171,48,35,156,...,212.8,192.2,147.5,131.2,172.2,237.2,244.2,243.2,234.2,236.2
0,196,192,165,72,93,178,190,92,25,150,...,125,116,166,233,248,229,177,104,146,225
1,98,186,162,104,62,157,186,130,20,142,...,151,216,246,236,161,64,43,184,249,186
2,236,143,170,139,61,120,148,150,24,114,...,232,226,173,66,33,142,216,238,255,149
3,250,227,171,125,76,112,123,147,41,88,...,121,71,37,88,202,232,204,229,255,110
4,158,244,168,77,83,93,134,142,82,61,...,41,74,160,237,212,200,244,255,252,75
5,127,184,160,65,78,71,146,145,126,48,...,171,191,211,192,200,250,231,165,163,56
6,179,155,168,110,91,69,144,167,178,65,...,137,178,194,229,242,163,63,95,147,38
7,141,160,177,144,136,81,109,171,192,118,...,142,147,167,151,87,73,165,246,164,35
8,156,165,184,152,155,122,88,138,163,155,...,86,89,98,118,175,224,231,224,111,39
9,153,188,170,144,143,146,92,110,134,157,...,134,127,125,142,159,198,219,240,79,30


In [47]:
# load traning and testing data
df_train = exn
df_test = exn
# df_test["filename"] = "clips-" + df_test["id"].astype(str) + ".jpg"
# df_train["clip_count"] = df_train["clip_count"].astype("float32")


In [49]:
# data preparation
data_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

train_dataset = ClipCountDataset(df_train, SOURCE, transform=data_transform)
val_dataset = ClipCountDataset(df_validate, SOURCE, transform=data_transform)
test_dataset = ClipCountDataset(df_test, SOURCE, transform=data_transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)


NameError: name 'transforms' is not defined

In [29]:
# Customize the layer of conversion from cone to 2D image
class cone2img(nn.Module):
    def __init__(self, in_features, out_features, mask):
        super(SparseLinear, self).__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.mask = mask

    def forward(self, x):
        masked_weight = self.weight * self.mask
        return torch.nn.functional.linear(x, masked_weight)

In [ ]:
# Customize the layer from cone2img to Conv2d 
class cone2img_Conv2d(nn.Module):
    def __init__(self, in_features: int, out_features: int, mask: torch.Tensor, conv_out_channels: int):
        super(cone2img_Conv2d, self).__init__()
        
        # Define SparseLinear layer
        self.sparse_linear = cone2img(in_features, out_features, mask)
        
        # Define Conv2d layer
        # Calculate the appropriate input channels for Conv2d
        self.reshape_height = 8  # Example value: needs to be compatible with your model
        self.reshape_width = 8   # Example value: needs to be compatible with your model
        self.conv1 = nn.Conv2d(in_channels=conv_out_channels, out_channels=32, kernel_size=3, padding=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Apply SparseLinear
        x = self.sparse_linear(x)
        
        # Reshape the output to fit the input for Conv2d
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.reshape_height, self.reshape_width)
        
        # Apply Conv2d
        x = self.conv1(x)
        return x

In [31]:
# generate CNN model
model = nn.Sequential(
    nn.cone2img(
    nn.Conv2d(3, 64, 3),  # 3 input channels, 64 output channels, 3x3 kernel
    nn.ReLU(),
    nn.MaxPool2d(2, 2),  # 2x2 pooling kernel with stride 2
    nn.Conv2d(64, 64, 3), # 64 input channels, 64 output channels, 3x3 kernel
    nn.ReLU(),
    nn.MaxPool2d(2, 2),  # 2x2 pooling kernel with stride 2
    nn.Flatten(),       # Flattening the tensor for the fully connected layers
    nn.Linear(64 * 62 * 62, 512), # 64 * 62 * 62 input features, 512 output features
    nn.ReLU(),
    nn.Linear(512, 1)    # 512 input features, 1 output feature
)
model = torch.compile(model,backend="aot_eager").to(device)


criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = ReduceLROnPlateau(optimizer, 'min')

SyntaxError: '(' was never closed (511398546.py, line 2)

In [ ]:
# training the model
EPOCHS = 1

print("Training")
for epoch in range(EPOCHS):
    running_loss = 0.0
    steps = list(enumerate(train_dataloader, 0))
    for i, data in tqdm.tqdm(steps):
        inputs, labels = data['image'].to(device).float(), data['clip_count'].to(device).float()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.view(-1), labels.view(-1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step(running_loss)
    print(f"Epoch {epoch}/{EPOCHS}, loss: {loss.item()}")

print('Finished Training')